# Anomaly detection using LSTM

This notebook is an example for how to use Long-Short Term Memory neural networks for anomaly detection.

Data used in this example is taken from hardware sensors attached to a robot arm performing simple tasks. We will be using 3 values - linear acceleration X/Y/Z. The goal is to predict robot malfunction before it crashes (and before a person is able to notice it).

Network graph will be prepared using Keras/Tensorflow and the training itself will be performed using DeepWater.

## Setup

Import required modules (h2o for deepwater and pandas for initial data munging).

Bootstrap a single node H2O instance.

In [ ]:
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator
import pandas as pd

PATH=os.path.expanduser("~/h2o-3")
h2o.init(port=54321, nthreads=-1)
if not H2ODeepWaterEstimator.available(): exit

## Custom TF LSTM model

Build a custom LSTM network graph using Keras and save it for later use.

In [ ]:
import tensorflow as tf
import json
from keras.layers.core import Dense, Flatten, Reshape
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import backend as K
from keras.objectives import categorical_crossentropy
from tensorflow.python.framework import ops

def keras_model(h, w):
    # always create a new graph inside ipython or
    # the default one will be used and can lead to
    # unexpected behavior
    graph = tf.Graph() 
    with graph.as_default():
        size = w * h
        # Input images fed via H2O
        inp = tf.placeholder(tf.float32, [None, size])
        # Actual labels used for training fed via H2O
        labels = tf.placeholder(tf.float32, [None, 1])

        # Keras network
        x = Reshape((w, h))(inp)

        x = LSTM(32, input_shape=(w, h))(x)
        x = LSTM(64, input_dim=64, input_length=10, return_sequences=True)(x)
        x = LSTM(32, return_sequences=True)(x)
        x = LSTM(10)(x)
        out = tf.reduce_sum(x)

        predictions = out

        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=out))
        train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

        init_op = tf.global_variables_initializer()

        # Metadata required by H2O
        tf.add_to_collection(ops.GraphKeys.INIT_OP, init_op.name)
        tf.add_to_collection(ops.GraphKeys.TRAIN_OP, train_step)
        tf.add_to_collection("logits", out)
        tf.add_to_collection("predictions", predictions)

        meta = json.dumps({
                "inputs": {"batch_image_input": inp.name,
                           "categorical_labels": labels.name},
                "outputs": {"categorical_logits": out.name,
                            "layers": ','.join([m.name for m in tf.get_default_graph().get_operations()])},
                "parameters": {}
            })
        tf.add_to_collection("meta", meta)

        # Save the meta file with the graph
        saver = tf.train.Saver()
        filename = "/tmp/keras_tf_lstm.meta"
        tf.train.export_meta_graph(filename, saver_def=saver.as_saver_def())

        return filename

## Load and prepare data

In [ ]:
# Read training data into memory
iot_raw = pd.from_csv('../model-builder/resources/normal_20170202_2229.csv')

# Select training columns
iot_train = iot_raw[[" LinAccX (g)"," LinAccY (g)"," LinAccZ (g)"]]

y = iot_train.sum(axis=1)
iot_train["y"] = y

# Send the data to H2O
iot_train_frame  = h2o.H2OFrame(iot)

## Train the model

In [ ]:
(h, w) = iot_train.shape
model_filename = keras_model(h, w-1)
model = H2ODeepWaterEstimator(epochs=500, 
                              network_definition_file=model_filename,
                              backend="tensorflow", 
                             ) 
model.train(x = [0:2], y = 3, training_frame=iot_train_frame)
model.show()